In [1]:
output_folder = '/home/jnaiman/data/morgan/'

ender = '_regular'

vocabulary_file = "data/vocabulary_new_pages_new"+ender+".pkl"
train_file = "data/train_aligned_new_pages"+ender+".pkl"
val_file = "data/dev_aligned_new_pages"+ender+".pkl"

window_length = 100 # for subsetting the data

In [2]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from nltk.lm import Vocabulary
import torch
import torch.utils.data as tud
# import sys
# sys.path.append("../../lib")
#from metrics import levenshtein
import pickle
from pathlib import Path
import re
import gc
import time
import glob

Read in data:

In [3]:
with open(output_folder+vocabulary_file, "rb") as file:
    vocabulary = pickle.load(file)

In [4]:
len(vocabulary)

585

In [5]:
train = pd.read_pickle(output_folder+train_file)
train.shape

(30922727, 2)

Char to integer/integer to char:

In [6]:
char2i = {c:i for i, c in enumerate(sorted(vocabulary), 3)}
char2i["<PAD>"] = 0
char2i["<START>"] = 1
char2i["<END>"] = 2
len(char2i)

588

In [7]:
i2char = {i:c for i, c in enumerate(sorted(vocabulary), 3)}
i2char[0] = "<PAD>"
i2char[1] = "<START>"
i2char[2] = "<END>"
len(i2char)

588

Padding sequences?

In [ ]:
#length = 100
#window_length = 100 # I think this is the same as length?

output = []
for s in tqdm(train.source):
    output.append(torch.tensor([1] + [char2i[c] for c in s] + [2]))
    
train_source = torch.nn.utils.rnn.pad_sequence(output, batch_first = True)
print(train_source.shape)

output = []
for s in tqdm(train.target):
    output.append(torch.tensor([1] + [char2i[c] for c in s] + [2]))
    
train_target = torch.nn.utils.rnn.pad_sequence(output, batch_first = True)
print(train_target.shape)

  0%|          | 0/30922727 [00:00<?, ?it/s]

Testing the padding maybe?

In [ ]:
train.source[0] == re.sub(r"<START>|<END>|<PAD>", "", "".join([i2char[c] for c in train_source[0].tolist()]))

In [ ]:
train.target[0] == re.sub(r"<START>|<END>|<PAD>", "", "".join([i2char[c] for c in train_target[0].tolist()]))

In [ ]:
torch.save(train_source, output_folder+"data/train_source_new_pages.pt")

In [ ]:
torch.save(train_target, output_folder+"data/train_target_new_pages.pt")

In [ ]:
del train

Same thing with the dev set:

In [ ]:
dev = pd.read_pickle(output_folder + val_file)
dev.shape

In [ ]:
output = []
for s in tqdm(dev.source):
    output.append(torch.tensor([1] + [char2i[c] for c in s] + [2]))
    
dev_source = torch.nn.utils.rnn.pad_sequence(output, batch_first = True)
print(dev_source.shape)

output = []
for s in tqdm(dev.target):
    output.append(torch.tensor([1] + [char2i[c] for c in s] + [2]))
    
dev_target = torch.nn.utils.rnn.pad_sequence(output, batch_first = True)
print(dev_target.shape)

In [ ]:
dev.source[0] == re.sub(r"<START>|<END>|<PAD>", "", "".join([i2char[c] for c in dev_source[0].tolist()]))

In [ ]:
dev.target[0] == re.sub(r"<START>|<END>|<PAD>", "", "".join([i2char[c] for c in dev_target[0].tolist()]))

Save all the things for training later:

In [ ]:
torch.save(dev_source, output_folder+"data/dev_source_new_pages.pt")

In [ ]:
torch.save(dev_target, output_folder+"data/dev_target_new_pages.pt")

In [ ]:
with open(output_folder+"data/char2i_new_pages.pkl", "wb") as file:
    pickle.dump(char2i, file)

In [ ]:
with open(output_folder+"data/i2char_new_pages.pkl", "wb") as file:
    pickle.dump(i2char, file)

# ------- orig after here ----------

In [3]:
folder = "./post_ocr_correction/data/en/data/"

In [4]:
train = pd.read_pickle(folder+"/"+"train_aligned_new_pages.pkl")
train.shape

(9011, 2)

In [5]:
dev = pd.read_pickle(folder+"/"+"dev_aligned.pkl")
dev.shape

(727, 2)

In [6]:
with open(folder+"/"+"vocabulary_new.pkl", "rb") as file:
    vocabulary = pickle.load(file)

In [5]:
#file_pattern = "train_aligned_new_full*.pkl"

In [6]:
#train_pickle_files = glob.glob(folder + "/" + file_pattern)

In [7]:
# dfs = []
# for pkl_file in train_pickle_files:
#     with open(pkl_file, 'rb') as f:
#         df = pickle.load(f)
#         dfs.append(df)

In [8]:
# df_all = pd.concat(dfs, axis=0)

In [7]:

batch_size = 1000

pickle_files = glob.glob(folder + "/" + file_pattern)

# Load each batch of pickle files into a list of dataframes
dfs_processed = []
for i in range(0, len(pickle_files), batch_size):
    dfs = []
    for pkl_file in pickle_files[i:i+batch_size]:
        with open(pkl_file, 'rb') as f:
            df = pickle.load(f)
            dfs.append(df)
    # Concatenate all of the dataframes in one dataframe batch
    df_batch = pd.concat(dfs, axis=0)
    # Append the dataframe batch to the list of dataframes
    dfs_processed.append(df_batch)

# Concatenate all of the batches into a single large dataframe
df_all = pd.concat(dfs_processed, axis=0)

NameError: name 'file_pattern' is not defined

In [10]:
train = df_all
train.shape

(91919976, 2)

In [11]:
# train = pd.read_pickle(folder/"train_aligned_new.pkl")
# train.shape

In [8]:
dev = pd.read_pickle(folder + "/" + "dev_aligned_new_full.pkl")
dev.shape

(635, 2)

In [9]:
with open(folder + "/" + "vocabulary_new.pkl", "rb") as file:
    vocabulary = pickle.load(file)

In [10]:
vocabulary.lookup('put')

'<UNK>'

In [11]:
len(vocabulary)

668

In [12]:
char2i = {c:i for i, c in enumerate(sorted(vocabulary), 3)}
char2i["<PAD>"] = 0
char2i["<START>"] = 1
char2i["<END>"] = 2
len(char2i)

671

In [13]:
i2char = {i:c for i, c in enumerate(sorted(vocabulary), 3)}
i2char[0] = "<PAD>"
i2char[1] = "<START>"
i2char[2] = "<END>"
len(i2char)

671

In [14]:
length = 100

output = []
for s in tqdm(train.source):
    output.append(torch.tensor([1] + [char2i[c] for c in s] + [2]))
    
train_source = torch.nn.utils.rnn.pad_sequence(output, batch_first = True)
print(train_source.shape)

output = []
for s in tqdm(train.target):
    output.append(torch.tensor([1] + [char2i[c] for c in s] + [2]))
    
train_target = torch.nn.utils.rnn.pad_sequence(output, batch_first = True)
print(train_target.shape)

  0%|          | 0/9011 [00:00<?, ?it/s]

torch.Size([9011, 80])


  0%|          | 0/9011 [00:00<?, ?it/s]

torch.Size([9011, 80])


In [15]:
train.source[0] == re.sub(r"<START>|<END>|<PAD>", "", "".join([i2char[c] for c in train_source[0].tolist()]))

True

In [16]:
train.target[0] == re.sub(r"<START>|<END>|<PAD>", "", "".join([i2char[c] for c in train_target[0].tolist()]))

True

In [59]:
#i2char

In [60]:
#char2i

In [48]:
# char2i[":"] = len(char2i)
# char2i["W"] = len(char2i)
# char2i["*"] = len(char2i)
# char2i["I"] = len(char2i)
# char2i["k"] = len(char2i)
# char2i["("] = len(char2i)
# char2i["~"] = len(char2i)
# char2i["B"] = len(char2i)
# char2i["O"] = len(char2i)

In [17]:
output = []
for s in tqdm(dev.source):
    output.append(torch.tensor([1] + [char2i[c] for c in s] + [2]))
    
dev_source = torch.nn.utils.rnn.pad_sequence(output, batch_first = True)
print(dev_source.shape)

output = []
for s in tqdm(dev.target):
    output.append(torch.tensor([1] + [char2i[c] for c in s] + [2]))
    
dev_target = torch.nn.utils.rnn.pad_sequence(output, batch_first = True)
print(dev_target.shape)

  0%|          | 0/635 [00:00<?, ?it/s]

torch.Size([635, 102])


  0%|          | 0/635 [00:00<?, ?it/s]

torch.Size([635, 102])


In [18]:
dev.source[0] == re.sub(r"<START>|<END>|<PAD>", "", "".join([i2char[c] for c in dev_source[0].tolist()]))

True

In [19]:
dev.target[0] == re.sub(r"<START>|<END>|<PAD>", "", "".join([i2char[c] for c in dev_target[0].tolist()]))

True

In [20]:
torch.save(train_source, folder+"/"+"train_source_new_pages.pt")

In [21]:
torch.save(train_target, folder+"/"+"train_target_new_pages.pt")

In [22]:
torch.save(dev_source, folder+"/"+"dev_source_new_pages.pt")

In [23]:
torch.save(dev_target, folder+"/"+"dev_target_new_pages.pt")

In [24]:
with open(folder+"/"+"char2i_new_pages.pkl", "wb") as file:
    pickle.dump(char2i, file)

In [25]:
with open(folder+"/"+"i2char_new_pages.pkl", "wb") as file:
    pickle.dump(i2char, file)

In [20]:
folder

'./post_ocr_correction/data/en/data/'

In [22]:
pickle_files = glob.glob(folder + "/" + file_pattern)

In [32]:
i_count = 0
train = ''
for p in pickle_files:
    del train
    time_start = time.time()
    with open(p, 'rb') as f:
        train = pickle.load(f)
#     print("done with read in", len(train))
        # Generate tensors for source
    batch_output_source = []
    for s in train.source:
        batch_output_source.append(torch.tensor([1] + [char2i[c] for c in s] + [2]))
#  print("done with source")
    # Generate tensors for target
    batch_output_target = []
    for s in train.target:
        batch_output_target.append(torch.tensor([1] + [char2i[c] for c in s] + [2]))
#     print("done with target") 
    train_source = torch.nn.utils.rnn.pad_sequence(batch_output_source, batch_first=True)
    train_target = torch.nn.utils.rnn.pad_sequence(batch_output_target, batch_first=True)
#     print("done with tensors")

    torch.save(train_source, folder+"train_source_new"+str(i_count).zfill(4)+".pt")
    torch.save(train_target, folder+"train_target_new"+str(i_count).zfill(4)+".pt")
    #print(i_count)
    i_count += 1
    del train_source
    del train_target
    del batch_output_target
    del batch_output_source
    
    
    
    gc.collect()
    
    end_time = time.time()
    print("On", i_count, "of", len(pickle_files), "in", end_time - time_start, "seconds")

On 1 of 92 in 45.022669315338135 seconds
On 2 of 92 in 44.53941750526428 seconds
On 3 of 92 in 45.85177445411682 seconds
On 4 of 92 in 44.169872522354126 seconds
On 5 of 92 in 44.02352285385132 seconds
On 6 of 92 in 44.474716901779175 seconds
On 7 of 92 in 44.30797505378723 seconds
On 8 of 92 in 43.99881410598755 seconds
On 9 of 92 in 44.1687376499176 seconds
On 10 of 92 in 45.215468406677246 seconds
On 11 of 92 in 45.70466995239258 seconds
On 12 of 92 in 44.08570861816406 seconds
On 13 of 92 in 44.3284113407135 seconds
On 14 of 92 in 44.312944412231445 seconds
On 15 of 92 in 44.34751105308533 seconds
On 16 of 92 in 44.12048316001892 seconds
On 17 of 92 in 44.28013849258423 seconds
On 18 of 92 in 44.58359146118164 seconds
On 19 of 92 in 44.67673921585083 seconds
On 20 of 92 in 44.10096311569214 seconds
On 21 of 92 in 44.14881420135498 seconds
On 22 of 92 in 44.15916895866394 seconds
On 23 of 92 in 44.510260820388794 seconds
On 24 of 92 in 44.21414589881897 seconds
On 25 of 92 in 44.681

In [ ]:
# batch_size = 1000

# pickle_files = glob.glob(folder + "/" + file_pattern)

# # Load each batch of pickle files into a list of dataframes
# for i in range(0, len(pickle_files), batch_size):
#     for pkl_file in pickle_files[i:i+batch_size]:
#         with open(pkl_file, 'rb') as f:
#             df = pickle.load(f)
#             dfs.append(df)
#     # Concatenate all of the dataframes in one dataframe batch
#     df_batch = pd.concat(dfs, axis=0)
#     # Append the dataframe batch to the list of dataframes
#     dfs_processed.append(df_batch)

# # Concatenate all of the batches into a single large dataframe
# df_all = pd.concat(dfs_processed, axis=0)

In [21]:

# length = 100

# output = []
# for s in tqdm(train.source):
#     output.append(torch.tensor([1] + [char2i[c] for c in s] + [2]))
    
# train_source = torch.nn.utils.rnn.pad_sequence(output, batch_first = True)
# print(train_source.shape)

# output = []
# for s in tqdm(train.target):
#     output.append(torch.tensor([1] + [char2i[c] for c in s] + [2]))
    
# train_target = torch.nn.utils.rnn.pad_sequence(output, batch_first = True)
# print(train_target.shape)


# batch_size = 10000
# output_source = []
# output_target = []

# # Split train.source and train.target into smaller chunks
# source_chunks = np.array_split(train.source, len(train.source) // batch_size + 1)
# target_chunks = np.array_split(train.target, len(train.target) // batch_size + 1)

# for i in range(len(source_chunks)):
#     # Get the current source and target chunk
#     batch_source = source_chunks[i]
#     batch_target = target_chunks[i]

#     # Generate tensors 
#     batch_output_source = []
#     batch_output_target = []
#     for s in tqdm(batch_source):
#         batch_output_source.append(torch.tensor([1] + [char2i[c] for c in s] + [2]))
#     for s in tqdm(batch_target):
#         batch_output_target.append(torch.tensor([1] + [char2i[c] for c in s] + [2]))

#     # Append batch tensors to the output lists
#     output_source.extend(batch_output_source)
#     output_target.extend(batch_output_target)

# #create the final tensors
# train_source = torch.nn.utils.rnn.pad_sequence(output_source, batch_first=True)
# train_target = torch.nn.utils.rnn.pad_sequence(output_target, batch_first=True)

# print(train_source.shape)
# print(train_target.shape)



# batch_size = 100

# # Split train.source and train.target into smaller chunks
# source_chunks = np.array_split(train.source, len(train.source) // batch_size + 1)
# target_chunks = np.array_split(train.target, len(train.target) // batch_size + 1)

# output_source = []
# output_target = []

# # Process each chunk separately
# i_count = 0
# for source_chunk, target_chunk in tqdm(zip(source_chunks, target_chunks), total=len(source_chunks)):
#     # Generate tensors for source
#     batch_output_source = []
#     for s in source_chunk:
#         batch_output_source.append(torch.tensor([1] + [char2i[c] for c in s] + [2]))

#     # Generate tensors for target
#     batch_output_target = []
#     for s in target_chunk:
#         batch_output_target.append(torch.tensor([1] + [char2i[c] for c in s] + [2]))
        
#     train_source = torch.nn.utils.rnn.pad_sequence(batch_output_source, batch_first=True)
#     train_target = torch.nn.utils.rnn.pad_sequence(batch_output_target, batch_first=True)

#     torch.save(train_source, folder/"train_source_new"+str(i_count).zfill(3)+".pt")
#     torch.save(train_target, folder/"train_target_new"+str(i_count).zfill(3)+".pt")
#     print(i_count)
#     i_count += 1
    
    
    # Append batch tensors to the output lists
#     output_source.extend(batch_output_source)
#     output_target.extend(batch_output_target)

#create the final tensors
# train_source = torch.nn.utils.rnn.pad_sequence(output_source, batch_first=True)
# train_target = torch.nn.utils.rnn.pad_sequence(output_target, batch_first=True)

# print(train_source.shape)
# print(train_target.shape)




KeyboardInterrupt: 

In [11]:
train.source[0] == re.sub(r"<START>|<END>|<PAD>", "", "".join([i2char[c] for c in train_source[0].tolist()]))

True

In [12]:
train.target[0] == re.sub(r"<START>|<END>|<PAD>", "", "".join([i2char[c] for c in train_target[0].tolist()]))

True

In [11]:
output = []
for s in tqdm(dev.source):
    output.append(torch.tensor([1] + [char2i[c] for c in s] + [2]))
    
dev_source = torch.nn.utils.rnn.pad_sequence(output, batch_first = True)
print(dev_source.shape)

output = []
for s in tqdm(dev.target):
    output.append(torch.tensor([1] + [char2i[c] for c in s] + [2]))
    
dev_target = torch.nn.utils.rnn.pad_sequence(output, batch_first = True)
print(dev_target.shape)

  0%|          | 0/635 [00:00<?, ?it/s]

torch.Size([635, 102])


  0%|          | 0/635 [00:00<?, ?it/s]

torch.Size([635, 102])


In [14]:
#dev.source[0] == re.sub(r"<START>|<END>|<PAD>", "", "".join([i2char[c] for c in dev_source[0].tolist()]))

True

In [15]:
#dev.target[0] == re.sub(r"<START>|<END>|<PAD>", "", "".join([i2char[c] for c in dev_target[0].tolist()]))

True

In [16]:
#torch.save(train_source, folder/"train_source_new.pt")

In [17]:
#torch.save(train_target, folder/"train_target_new.pt")

In [13]:
torch.save(dev_source, folder + "/"+ "dev_source_new_full.pt")

In [14]:
torch.save(dev_target, folder+"/"+"dev_target_new_full.pt")

In [16]:
with open(folder+"/"+"char2i_new_full.pkl", "wb") as file:
    pickle.dump(char2i, file)

In [17]:
with open(folder+"/"+"i2char_new_full.pkl", "wb") as file:
    pickle.dump(i2char, file)

In [22]:
#train_source.shape

torch.Size([962857, 102])